## Import Packages

In [1]:
import pandas as pd
import json

## Task 1

Insert 4 vehicles from the State College Auto Database (can be the same ones you used in Redis) into the MongoDB.  One easy way to do this is to create a .json file from the data and then import the .json file into a new data collection using the 'mongoimport' package as you did in HW #3 to import the restaurant dataset. 

In [14]:
cars = pd.read_csv('cars.csv',sep=',',index_col=0)
with open('cars.json', 'w') as outfile:
    cars_dict = cars.to_dict('index')
    for i in range(len(cars_dict)):
        json.dump(cars_dict[i], outfile)

In [15]:
! mongoimport --db local --collection cars --file cars.json

2018-11-06T01:22:33.494-0500	connected to: localhost
2018-11-06T01:22:34.035-0500	imported 10 documents


![](1.png)

## Task 2

Query your database and demonstrate that you can search on at least 2 attributes of the vehicles. Demonstrate a query on a single attribute and a query on more than one attribute. 

- query on a single attribute
![](2.png)

- query on more than one attribute
![](3.png)




## Task 3

Demonstrate a query for a vehicle not in the database. 

![](4.png)

## Task 4

What are differences you found between using MongoDB and Redis for loading and storing this data?  What would be a role for a database like Redis in storing this vehicle information? Can you imagine challenges in using the method you used for data import into MongoDB when dealing with Big Data?  How might you address any challenges in a big data environment? 

With mongoDB we can import the whole dataset in the json format directly while in Redis, we have to import one key-value pair at a time. In this case, Redis acts like a dictionary in python used to store the nested dictionaries (key-value pairs). 

In [3]:
from pyspark.sql import SparkSession, Row, functions

def parseInput(line):
    fields = line.split('|')
    return Row(user_id = int(fields[0]), 
               age = int(fields[1]),
               gender = fields[2],
               occupation = fields[3],
               zip = fields[4])

if __name__ == '__main__':
    spark = SparkSession.builder.appName('MongoDBIntegration').getOrCreate()
    lines = spark.sparkContext.textFile('hdfs://localhost:9002/user/data/u.user')
    users = lines.map(parseInput)
    usersDataset = spark.createDataFrame(users)
    
    # write into MongoDB
    usersDataset.write \
                .format('com.mongodb.spark.sql.DefaultSource') \
                .option('uri', 'mongodb://localhost:27017/movie.users') \
                .mode('append') \
                .save()
    

Name: org.apache.toree.interpreter.broker.BrokerException
Message: Py4JJavaError: An error occurred while calling o188.save.
: java.lang.ClassNotFoundException: Failed to find data source: com.mongodb.spark.sql.DefaultSource. Please find packages at http://spark.apache.org/third-party-projects.html
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:546)
	at org.apache.spark.sql.execution.datasources.DataSource.providingClass$lzycompute(DataSource.scala:87)
	at org.apache.spark.sql.execution.datasources.DataSource.providingClass(DataSource.scala:87)
	at org.apache.spark.sql.execution.datasources.DataSource.write(DataSource.scala:467)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:50)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:58)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala